In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# This line makes matplotlib plots appear inline in the notebook
%matplotlib inline


In [ ]:
# Define the ticker and time period
ticker = 'AAPL'
start_date = '2015-01-01'
end_date = '2023-01-01'

# Download historical data using yfinance
data = yf.download(ticker, start=start_date, end=end_date)

# Display the first few rows of the data
data.head()


In [ ]:
# Define window sizes for the short and long moving averages
short_window = 40
long_window = 100

# Calculate the short-term and long-term Simple Moving Averages (SMAs)
data['SMA_short'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
data['SMA_long'] = data['Close'].rolling(window=long_window, min_periods=1).mean()

# Display the updated DataFrame
data[['Close', 'SMA_short', 'SMA_long']].head(10)


In [ ]:
# Create a new column 'Signal' that is 1 when the short SMA is above the long SMA and 0 otherwise
data['Signal'] = np.where(data['SMA_short'] > data['SMA_long'], 1.0, 0.0)

# Create a 'Position' column to identify the points where a trade is initiated (i.e. a change in signal)
data['Position'] = data['Signal'].diff()

# Display the signals
data[['Close', 'SMA_short', 'SMA_long', 'Signal', 'Position']].tail(10)


In [ ]:
plt.figure(figsize=(14,7))
plt.plot(data['Close'], label=ticker, alpha=0.5)
plt.plot(data['SMA_short'], label=f'SMA {short_window}', alpha=0.75)
plt.plot(data['SMA_long'], label=f'SMA {long_window}', alpha=0.75)

# Plot buy signals (when Position == 1.0)
plt.plot(data.loc[data['Position'] == 1.0].index,
         data['SMA_short'][data['Position'] == 1.0],
         '^', markersize=10, color='g', label='Buy Signal')

# Plot sell signals (when Position == -1.0)
plt.plot(data.loc[data['Position'] == -1.0].index,
         data['SMA_short'][data['Position'] == -1.0],
         'v', markersize=10, color='r', label='Sell Signal')

plt.title(f'{ticker} Price with Buy/Sell Signals')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()


In [ ]:
# Calculate the daily percentage change (market returns)
data['Market Return'] = data['Close'].pct_change()

# Compute the strategy returns:
# We shift the signal by one day to avoid look-ahead bias (i.e., acting on tomorrow's information).
data['Strategy Return'] = data['Market Return'] * data['Signal'].shift(1)

# Compute the cumulative returns for the market and the strategy
data['Cumulative Market Return'] = (1 + data['Market Return']).cumprod()
data['Cumulative Strategy Return'] = (1 + data['Strategy Return']).cumprod()

# Plot the cumulative returns
plt.figure(figsize=(14,7))
plt.plot(data['Cumulative Market Return'], label='Market Return', alpha=0.75)
plt.plot(data['Cumulative Strategy Return'], label='Strategy Return', alpha=0.75)
plt.title('Cumulative Returns: Market vs. Moving Average Crossover Strategy')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.show()


In [ ]:
print("Final Cumulative Market Return:", data['Cumulative Market Return'].iloc[-1])
print("Final Cumulative Strategy Return:", data['Cumulative Strategy Return'].iloc[-1])